Question1: How much impact does being late or too spaced out at the first stop have downstream?

In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
wego = pd.read_csv('../data/headway_data_clean.csv')

In [3]:
# convert times to datetime64 objects, check dtype to confirm
wego['SCHEDULED_TIME'] = pd.to_datetime(wego['SCHEDULED_TIME'], format = '%Y-%m-%d %H:%M:%S')
wego['ACTUAL_ARRIVAL_TIME'] = pd.to_datetime(wego['ACTUAL_ARRIVAL_TIME'], format = '%Y-%m-%d %H:%M:%S')
wego['ACTUAL_DEPARTURE_TIME'] = pd.to_datetime(wego['ACTUAL_DEPARTURE_TIME'], format = '%Y-%m-%d %H:%M:%S')
wego['DATE'] = pd.to_datetime(wego['DATE'], format = "%Y/%m/%d")


In [4]:
#Create a column that gives each day of the week its on variable
wego['DAYS_OF_THE_WEEK'] = wego['DATE'].dt.weekday


In [5]:
#Lateness information on the first stop, middle stops and the last stop
wego.groupby('TRIP_EDGE')['ADHERENCE'].describe()

,count,mean,std,min,25%,50%,75%,max
TRIP_EDGE,,,,,,,,
0,207872.0,-3.692178,5.634956,-141.183333,-5.016666,-2.30,-0.583333,85.666666
1,65711.0,-2.760552,8.510429,-948.533333,-3.533333,-1.95,-0.833333,84.666666
2,65277.0,-2.016883,8.407097,-489.316666,-4.433333,-0.50,2.333333,88.383333


In [67]:
wego = wego.dropna(subset = 'ADHERENCE')

In [63]:
wego['row_num'] = 1
wego['STOPS'] = wego.groupby(['TRIP_ID', 'CALENDAR_ID'])['row_num'].cumsum()

In [46]:
wego.groupby(['ROUTE_ABBR', 'TRIP_EDGE'])['ADHERENCE'].mean()

ROUTE_ABBR  TRIP_EDGE
3           0           -3.958963
            1           -3.028399
            2           -3.809365
7           0           -4.184212
            1           -2.432742
            2           -3.432767
22          0           -2.303106
            1           -1.758424
            2            1.090347
23          0           -2.273851
            1           -2.630511
            2            0.566829
50          0           -2.885701
            1           -2.690048
            2           -0.845558
52          0           -4.143715
            1           -3.435693
            2           -3.852329
55          0           -5.233580
            1           -3.382513
            2           -4.654291
56          0           -3.260917
            1           -2.426649
            2           -0.781248
Name: ADHERENCE, dtype: float64

In [71]:
wego.groupby(['ROUTE_ABBR', 'STOPS', 'ROUTE_DIRECTION_NAME'])['ADHERENCE'].mean().sort_index()

ROUTE_ABBR  STOPS  ROUTE_DIRECTION_NAME
3           1      FROM DOWNTOWN          -3.238173
                   TO DOWNTOWN            -2.810734
            2      FROM DOWNTOWN          -4.921646
                   TO DOWNTOWN            -2.621232
            3      FROM DOWNTOWN          -5.066334
                                             ...   
56          10     TO DOWNTOWN            -3.983333
            11     FROM DOWNTOWN          -8.065432
            12     FROM DOWNTOWN          -7.573809
            13     FROM DOWNTOWN          -7.877333
            14     FROM DOWNTOWN          -5.765686
Name: ADHERENCE, Length: 174, dtype: float64

In [70]:
wego.groupby(['ROUTE_ABBR', 'STOPS', 'ROUTE_DIRECTION_NAME'])['ADHERENCE'].mean().sort_index().tail(10)

ROUTE_ABBR  STOPS  ROUTE_DIRECTION_NAME
56          8      FROM DOWNTOWN          -7.274359
                   TO DOWNTOWN            -6.463095
            9      FROM DOWNTOWN          -8.189881
                   TO DOWNTOWN            -4.438889
            10     FROM DOWNTOWN          -8.172619
                   TO DOWNTOWN            -3.983333
            11     FROM DOWNTOWN          -8.065432
            12     FROM DOWNTOWN          -7.573809
            13     FROM DOWNTOWN          -7.877333
            14     FROM DOWNTOWN          -5.765686
Name: ADHERENCE, dtype: float64

In [65]:
wego.loc[
    (wego['ROUTE_ABBR'] == 56) & (wego['STOPS']== 20)
    
    
]

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,TRIP_EDGE,LATITUDE,LONGITUDE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,DAYS_OF_THE_WEEK,row_num,stops,STOPS
325798,120230926,1,100285789,2023-09-26,56,5606,1389,353036,2,FROM DOWNTOWN,SAMS,12.0,0,36.307973,-86.685963,2023-09-26 13:07:00,NaT,NaT,NaN,10.0,NaN,NaN,0,0,0,0,1.0,0,279,NaN,1,1,20,20


In [6]:
#Creating a dataframe of the info I will need to answer the question
trip_adherence = wego[['TRIP_ID','CALENDAR_ID', 'TRIP_EDGE', 'ADHERENCE']]

In [40]:
trip_adherence = trip_adherence.copy()

In [41]:
trip_adherence[(trip_adherence['TRIP_EDGE'] == 1) & (trip_adherence['ADHERENCE'] < 0)].shape 

(59602, 5)

In [42]:
trip_adherence[trip_adherence['TRIP_EDGE'] == 1].shape

(67415, 5)

In [10]:
#How often a bus route will begin late
59602/67415

0.8841059111473708

**88% of the time a bus will begin late **

In [43]:
trip_adherence

,TRIP_ID,CALENDAR_ID,TRIP_EDGE,ADHERENCE,TRIP_IDS
0,345104,120230801,1,-2.133333,345104120230801
1,345104,120230801,0,-2.450000,345104120230801
2,345104,120230801,0,-0.933333,345104120230801
3,345104,120230801,2,6.283333,345104120230801
4,345105,120230801,1,-1.583333,345105120230801
...,...,...,...,...,...
350323,353448,120230930,0,-8.433333,353448120230930
350324,353448,120230930,2,-11.300000,353448120230930
350325,353449,120230930,1,-4.316666,353449120230930
350326,353449,120230930,0,-22.083333,353449120230930


In [13]:
#changing the dtype to string so that i can concat with the "CALENDAR ID" column
trip_adherence['TRIP_ID'] = trip_adherence['TRIP_ID'].astype(str)


In [14]:
#changing the dtype to string so that i can concat with the "ROUTE ID" column
trip_adherence['CALENDAR_ID'] = trip_adherence['CALENDAR_ID'].astype(str)


In [44]:
#creating a column that concats the two columns
trip_adherence['TRIP_IDS'] = trip_adherence['TRIP_ID'] + trip_adherence['CALENDAR_ID']
trip_adherence

,TRIP_ID,CALENDAR_ID,TRIP_EDGE,ADHERENCE,TRIP_IDS
0,345104,120230801,1,-2.133333,345104120230801
1,345104,120230801,0,-2.450000,345104120230801
2,345104,120230801,0,-0.933333,345104120230801
3,345104,120230801,2,6.283333,345104120230801
4,345105,120230801,1,-1.583333,345105120230801
...,...,...,...,...,...
350323,353448,120230930,0,-8.433333,353448120230930
350324,353448,120230930,2,-11.300000,353448120230930
350325,353449,120230930,1,-4.316666,353449120230930
350326,353449,120230930,0,-22.083333,353449120230930


In [22]:
trips = trip_adherence.pivot_table(values = 'ADHERENCE', index = 'TRIP_IDS', columns = 'TRIP_EDGE').reset_index(drop = True )

In [24]:
trips

TRIP_EDGE,0,1,2
0,-1.691667,-2.133333,6.283333
1,-1.658333,-2.450000,8.016666
2,-0.641667,-0.766666,6.200000
3,0.950000,-1.583333,1.500000
4,-2.650000,-3.116666,-1.916666
...,...,...,...
66445,NaN,0.966666,2.200000
66446,NaN,-0.266666,-139.683333
66447,NaN,0.966666,-166.566666
66448,NaN,-2.566666,-5.816666


In [45]:
trip_adherence.groupby(['TRIP_IDS', 'TRIP_EDGE']).mean()

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_30846/3859312826.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  trip_adherence.groupby(['TRIP_IDS', 'TRIP_EDGE']).mean()


ADHERENCE
TRIP_IDS        TRIP_EDGE            
345104120230801 0           -1.691667
                1           -2.133333
                2            6.283333
345104120230802 0           -1.658333
                1           -2.450000
...                               ...
354106120230927 2         -166.566666
354106120230928 1           -2.566666
                2           -5.816666
354106120230929 1          -20.616666
                2          -15.933333

[201636 rows x 1 columns]

In [32]:
wego

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,TRIP_EDGE,LATITUDE,LONGITUDE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,DAYS_OF_THE_WEEK
0,120230801,1,99457890,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,MHSP,14.0,1,36.181248,-86.847705,2023-08-01 04:42:00,2023-08-01 04:37:38,2023-08-01 04:44:08,-2.133333,NaN,NaN,NaN,0,0,1,0,0.0,0,2,6.500000,1
1,120230801,1,99457891,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,ELIZ,10.0,0,36.193454,-86.839981,2023-08-01 04:46:00,2023-08-01 04:48:27,2023-08-01 04:48:27,-2.450000,NaN,NaN,NaN,0,0,1,0,0.0,0,9,0.000000,1
2,120230801,1,99457892,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,CV23,5.0,0,36.182177,-86.814445,2023-08-01 04:54:00,2023-08-01 04:54:56,2023-08-01 04:54:56,-0.933333,NaN,NaN,NaN,0,0,1,0,0.0,0,19,0.000000,1
3,120230801,1,99457893,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,MCC5_10,1.0,2,36.167091,-86.781923,2023-08-01 05:10:00,2023-08-01 05:03:43,2023-08-01 05:03:43,6.283333,NaN,NaN,NaN,0,0,1,0,NaN,0,35,0.000000,1
4,120230801,1,99457894,2023-08-01,22,2200,1040,345105,0,FROM DOWNTOWN,MCC5_10,1.0,1,36.167091,-86.781923,2023-08-01 05:15:00,2023-08-01 05:03:43,2023-08-01 05:16:35,-1.583333,NaN,NaN,NaN,0,0,1,0,0.0,0,36,12.866666,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350323,120230930,2,100406610,2023-09-30,7,702,2312,353448,0,TO DOWNTOWN,21BK,4.0,0,36.138372,-86.800622,2023-09-30 22:23:00,2023-09-30 22:31:26,2023-09-30 22:31:26,-8.433333,30.0,35.266666,5.266666,0,1,0,0,0.0,0,539,0.000000,5
350324,120230930,2,100406611,2023-09-30,7,702,2312,353448,0,TO DOWNTOWN,MCC5_9,3.0,2,36.167091,-86.781923,2023-09-30 22:38:00,2023-09-30 22:49:18,2023-09-30 22:49:18,-11.300000,NaN,NaN,NaN,0,1,0,0,NaN,0,551,0.000000,5
350325,120230930,2,100406612,2023-09-30,7,702,2312,353449,0,FROM DOWNTOWN,MCC5_9,3.0,1,36.167091,-86.781923,2023-09-30 22:45:00,2023-09-30 22:49:19,2023-09-30 22:49:19,-4.316666,30.0,31.866666,1.866666,0,0,1,0,0.0,0,552,0.000000,5
350326,120230930,2,100406613,2023-09-30,7,702,2312,353449,0,FROM DOWNTOWN,21BK,4.0,0,36.138881,-86.800622,2023-09-30 22:59:00,2023-09-30 23:21:05,2023-09-30 23:21:05,-22.083333,30.0,47.500000,17.500000,0,1,0,0,0.0,0,563,0.000000,5


**Question does the routes with more stops have a correlation with adherence**

In [47]:
wego.

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,TRIP_EDGE,LATITUDE,LONGITUDE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,DAYS_OF_THE_WEEK
0,120230801,1,99457890,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,MHSP,14.0,1,36.181248,-86.847705,2023-08-01 04:42:00,2023-08-01 04:37:38,2023-08-01 04:44:08,-2.133333,NaN,NaN,NaN,0,0,1,0,0.0,0,2,6.500000,1
1,120230801,1,99457891,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,ELIZ,10.0,0,36.193454,-86.839981,2023-08-01 04:46:00,2023-08-01 04:48:27,2023-08-01 04:48:27,-2.450000,NaN,NaN,NaN,0,0,1,0,0.0,0,9,0.000000,1
2,120230801,1,99457892,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,CV23,5.0,0,36.182177,-86.814445,2023-08-01 04:54:00,2023-08-01 04:54:56,2023-08-01 04:54:56,-0.933333,NaN,NaN,NaN,0,0,1,0,0.0,0,19,0.000000,1
3,120230801,1,99457893,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,MCC5_10,1.0,2,36.167091,-86.781923,2023-08-01 05:10:00,2023-08-01 05:03:43,2023-08-01 05:03:43,6.283333,NaN,NaN,NaN,0,0,1,0,NaN,0,35,0.000000,1
4,120230801,1,99457894,2023-08-01,22,2200,1040,345105,0,FROM DOWNTOWN,MCC5_10,1.0,1,36.167091,-86.781923,2023-08-01 05:15:00,2023-08-01 05:03:43,2023-08-01 05:16:35,-1.583333,NaN,NaN,NaN,0,0,1,0,0.0,0,36,12.866666,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350323,120230930,2,100406610,2023-09-30,7,702,2312,353448,0,TO DOWNTOWN,21BK,4.0,0,36.138372,-86.800622,2023-09-30 22:23:00,2023-09-30 22:31:26,2023-09-30 22:31:26,-8.433333,30.0,35.266666,5.266666,0,1,0,0,0.0,0,539,0.000000,5
350324,120230930,2,100406611,2023-09-30,7,702,2312,353448,0,TO DOWNTOWN,MCC5_9,3.0,2,36.167091,-86.781923,2023-09-30 22:38:00,2023-09-30 22:49:18,2023-09-30 22:49:18,-11.300000,NaN,NaN,NaN,0,1,0,0,NaN,0,551,0.000000,5
350325,120230930,2,100406612,2023-09-30,7,702,2312,353449,0,FROM DOWNTOWN,MCC5_9,3.0,1,36.167091,-86.781923,2023-09-30 22:45:00,2023-09-30 22:49:19,2023-09-30 22:49:19,-4.316666,30.0,31.866666,1.866666,0,0,1,0,0.0,0,552,0.000000,5
350326,120230930,2,100406613,2023-09-30,7,702,2312,353449,0,FROM DOWNTOWN,21BK,4.0,0,36.138881,-86.800622,2023-09-30 22:59:00,2023-09-30 23:21:05,2023-09-30 23:21:05,-22.083333,30.0,47.500000,17.500000,0,1,0,0,0.0,0,563,0.000000,5
